In [1]:
# Run this code once 
# !pip install newsapi-python newspaper3l

In [2]:
from newsapi import NewsApiClient
from newspaper import Article
import credentials
import pickle
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, cpu_count
from sys import getsizeof

In [3]:
keywords = ['riot','riots','clash','clashed','clashes','violence', 'violent','election', 'elections', 'politician', 'politics', 'government', 'policy', 'rule', 'law', 'policies', 'rule', 'killed', 'kill', 'demonstration','India','Indonesia','Thailand', 'threatened', 'hate']
sources = ['the-hindu', 'google-news-in', 'the-times-of-india']

In [4]:
def getArticle(url, return_text_only = True):
    article = Article(url)
    article.download()
    article.parse()
    if return_text_only:
        return article.text
    return article

In [5]:
dataset = []
try:
    dataset = pickle.load(open('captured_news_dump.csv','rb'))
except:
    newsapi = NewsApiClient(credentials.retrieve_credential('news_api','aashish'))
    per_page = 100
    max_pages_allowed = 11
    for k in keywords:
        query_result = newsapi.get_everything(q = k, page_size=per_page, from_param='2019-02-21', sort_by='publishedAt', sources=','.join(sources), page = 1)
        dataset.append(query_result)
        print('looking up for ', k, 'found = ', query_result['totalResults'])
        for n in range(2, min(query_result['totalResults']//100 + 1, max_pages_allowed)):
            print('fetching page = ', n)
            query_result = newsapi.get_everything(q = k, page_size=per_page, from_param='2019-02-21', sort_by='publishedAt', sources=','.join(sources), page = n)
            dataset.append(query_result)
        print('Done')
    pickle.dump(dataset, open('captured_news_dump.csv','wb'))

In [6]:
new_dataset = []

In [7]:
for n in range(len(dataset)):
    new_dataset.extend(dataset[n]['articles'])

In [8]:
without_hindu = []
for n in range(len(new_dataset)):
    if 'hindu' in new_dataset[n]['url']:
        continue
    without_hindu.append(new_dataset[n])

In [9]:
del new_dataset, dataset

In [10]:
len(without_hindu)

7513

In [11]:
# pickle.dump(new_dataset, open('list_of_articles.pkl','wb'))

In [12]:
def fetch_news(article):
    try:
        article['content'] = getArticle(article['url'])
        article['fetched'] = True
    except:
        pass
    return article
    
nums = 30000
full_news = []
with Pool(cpu_count()) as p:
#           r = list(tqdm(p.imap(_foo, range(30)), total=30))
    full_news = list(tqdm(p.imap(fetch_news, without_hindu), total=len(without_hindu)))

In [13]:
!say "Hey, Aashish. It's Done"

In [14]:
getsizeof(full_news)

67728

In [17]:
# pickle.dump(full_news, open('news_with_content.pkl','wb'))